In [27]:
from transformers import AutoTokenizer  # Or BertTokenizer
from transformers import AutoModelForPreTraining  # Or BertForPreTraining for loading pretraining heads
from transformers import AutoModel  # or BertModel, for BERT without pretraining heads
import torch
from transformers import BertTokenizer

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_classif
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [143]:
authors = pd.read_csv("../data/authors.csv")
authors = authors[authors.username.isin(["MidnightRider00", "TheGza1"])]
authors = authors[["comment", "username"]]
authors = shuffle(authors, random_state=42)
authors.reset_index(inplace=True)
authors.drop("index", axis=1, inplace=True)

In [144]:
tokenizer = BertTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
labels = {
    'MidnightRider00':0,
    'TheGza1':1
}

In [145]:
y = [labels[label] for label in authors['username']]
print("Quantidade de comentários:", len(y))

Quantidade de comentários: 2021


In [146]:
texts = [tokenizer.encode(text, 
                padding='max_length', 
                max_length = 512, 
                truncation=True, 
                return_tensors="pt") 
            for text in authors['comment']]

In [147]:
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
bert_embeddings = list()

for i, text in enumerate(texts):
    with torch.no_grad():
        outs = model(text)
        encoded = outs[0][0, 1:-1]
        bert_embeddings.append(encoded)
    if i%100 == 0: print(i)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0
100
200
300
400
500
600


In [ ]:
X = bert_embeddings
y = authors["username"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [91]:
#X_train2 = [np.array(x.reshape(1,-1)) for x in X_train]
#len(X_train2)

67

In [ ]:
X_train2 = np.array([np.array(x.reshape(1,-1).flatten()) for x in X_train])
X_train2.shape

(67, 391680)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train2, y_train)

DecisionTreeClassifier()

In [ ]:
X_test2 = np.array([np.array(x.reshape(1,-1).flatten()) for x in X_test])
X_test2.shape

(34, 391680)

In [ ]:
y_pred = clf.predict(X_test2)

In [ ]:
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

MidnightRider00       0.68      0.68      0.68        22
        TheGza1       0.42      0.42      0.42        12

       accuracy                           0.59        34
      macro avg       0.55      0.55      0.55        34
   weighted avg       0.59      0.59      0.59        34

